In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("tour.csv")

In [3]:
df.info() # null values for 'Rating' and 'Number of reviews' bc tour items at the end tend to not have any ratings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Tour Name          984 non-null    object
 1   Type               983 non-null    object
 2   Rating             485 non-null    object
 3   Number of reviews  485 non-null    object
 4   Price              948 non-null    object
 5   Duration           984 non-null    object
dtypes: object(6)
memory usage: 46.2+ KB


In [4]:
df.head()

,Tour Name,Type,Rating,Number of reviews,Price,Duration
0,1. 1 Day Private Mt Fuji Tour (Charter) - Engl...,Cable Car Tours,5.0 of 5 bubbles,159,$473.38,6+ hours
1,2. Mt Fuji and Hakone 1-Day Bus Tour return by...,Audio Guides,5.0 of 5 bubbles,"1,292",$141.09,6+ hours
2,3. Tokyo Full-Day Private Tour with Government...,Public Transportation Tours,5.0 of 5 bubbles,571,$152.04,6 hours
3,"4. Mt Fuji, Hakone Lake Ashi Cruise Bullet Tra...",Bus Tours,4.0 of 5 bubbles,652,$133.94,6+ hours
4,5. Private Full Day Sightseeing Tour to Mount ...,Bus Tours,5.0 of 5 bubbles,286,$647.98,6+ hours


## Preprocessing

In [5]:
# get rid of number. string in front of Tour Name column values
df['Tour Name'] = df['Tour Name'].str.replace(r'^\d+\. ', '', regex = True)

In [6]:
# get rid of 'of bubbles' string from Rating column values
# then convert to float
df['Rating'] = df['Rating'].str.replace('of 5 bubbles', '')
df['Rating'] = df['Rating'].astype(float)

In [7]:
# convert 'Number of reviews' to int type
df['Number of reviews'] = df['Number of reviews'].str.replace(',', '')
df['Number of reviews'] = df['Number of reviews'].astype('Int64') # to take NaN into account as well

In [8]:
df['Price'] = df['Price'].str.replace('[$,]', '', regex = True).astype(float)

In [9]:
df

,Tour Name,Type,Rating,Number of reviews,Price,Duration
0,1 Day Private Mt Fuji Tour (Charter) - English...,Cable Car Tours,5.0,159,473.38,6+ hours
1,Mt Fuji and Hakone 1-Day Bus Tour return by Bu...,Audio Guides,5.0,1292,141.09,6+ hours
2,Tokyo Full-Day Private Tour with Government-Li...,Public Transportation Tours,5.0,571,152.04,6 hours
3,"Mt Fuji, Hakone Lake Ashi Cruise Bullet Train ...",Bus Tours,4.0,652,133.94,6+ hours
4,Private Full Day Sightseeing Tour to Mount Fuj...,Bus Tours,5.0,286,647.98,6+ hours
...,...,...,...,...,...,...
979,"Private Lunch with Sayuki, the First Western G...",Transportation Services,NaN,<NA>,NaN,2 hours
980,Transfer in private vehicle from Tokyo City Ce...,Full-day Tours,NaN,<NA>,NaN,1–2 hours
981,Private Custom Day Tour in Tokyo City,Transportation Services,NaN,<NA>,NaN,6+ hours
982,Tokyo Private Transfer from Tokyo central to N...,Food & Drink,NaN,<NA>,NaN,1 hour


In [10]:
duration_val = df['Duration'].unique()
print(duration_val)

['6+ hours' '6 hours' '1–2 hours' '9–10 hours' '5–6 hours' '3 hours'
 '2 hours' '7–9 hours' '2–3 hours' '4–8 hours' '3–8 hours' '6–7 hours'
 '4 hours' '60–90 minutes' '3–4 hours' '4–5 hours' '8–9 hours'
 '30–50 minutes' '3–6 hours' '8–10 hours' '10–12 hours' '2–8 hours'
 '1 hour' '30–60 minutes' '1–10 hours' '10–11 hours' '1–2 days'
 '4–10 hours' '4–6 hours' '3+ days' '1–24 hours' '45–60 minutes'
 '2–6 hours' '1–3 hours' '30–180 minutes' '5 hours' '60–150 minutes'
 '7–8 hours' '3–30 days' '6–8 hours' '2 days' '12–16 hours' '45 minutes'
 '4–7 hours' '180–195 minutes' '30 minutes' '25 minutes' '3–5 hours'
 '50–70 minutes' '150–180 minutes' '13–14 hours' '30–120 minutes'
 '80–100 minutes' '11–12 hours' '5–7 hours' '50 minutes' '5–10 hours'
 '2–5 hours' '5–8 hours' '3 days' '40 minutes' '2–4 hours'
 '270–300 minutes' '6–10 hours' '120–150 minutes' '4–12 hours'
 '120–360 minutes' '60–120 minutes' '1 day' '20–90 minutes'
 '90–120 minutes' '9–11 hours' '3–12 hours' '2–4 minutes' '11–13 hours'

In [11]:
df[df['Duration'] == '11–13 hours']

,Tour Name,Type,Rating,Number of reviews,Price,Duration
897,Private Full-Day Guided Tour in Mount Fuji Lakes,Transportation Services,NaN,<NA>,253.4,11–13 hours


From the above result showing all unique values for 'Duration' column, some conversions seem necessary. I'd like to make hours as the standard, and convert all other measures to hours. Additionally, since most of the values are in the form of approximate range (e.g. 7-9 hours), I'll take the average for such values (e.g. 8 hours).

In [12]:
df_hours = df[df['Duration'].str.contains('hours|hour')].copy() # '.copy()' to create a copy of the filtered df and avoid warning
hours_val = df_hours['Duration'].str.extract(r'(\d+(?:–\d+)?)', expand=False).str.split('–')
# for values such as '6-8 hours', calculate the average of lower and upper bound
hours_val = hours_val.apply(lambda x: sum([float(i) for i in x])/len(x) if len(x) > 1 else float(x[0]))
df_hours['Duration'] = hours_val.round(1) # round to one decimal point

Accessing the same row (index no.897) shows that the average hour has been calculated correctly!

In [13]:
df_hours[df_hours['Tour Name'] == 'Private Full-Day Guided Tour in Mount Fuji Lakes']

,Tour Name,Type,Rating,Number of reviews,Price,Duration
897,Private Full-Day Guided Tour in Mount Fuji Lakes,Transportation Services,NaN,<NA>,253.4,12.0


In [14]:
df_minutes = df[df['Duration'].str.contains('minutes')].copy()
minutes_val = df_minutes['Duration'].str.extract(r'(\d+(?:–\d+)?)', expand=False).str.split('–')
# for values such as '40-60 minutes', calculate the average of lower and upper bound
minutes_val = minutes_val.apply(lambda x: sum([float(i) for i in x])/len(x) if len(x) > 1 else float(x[0]))
# convert minutes to hours and round to one decimal point
minutes_val = minutes_val/60
df_minutes['Duration'] = minutes_val.round(1)

In [15]:
df_days = df[df['Duration'].str.contains('days|day')].copy()
days_val = df_days['Duration'].str.extract(r'(\d+(?:–\d+)?)', expand=False).str.split('–')
days_val = days_val.apply(lambda x: sum([float(i) for i in x])/len(x) if len(x) > 1 else float(x[0]))
# convert days to hours
days_val = days_val * 24
df_days['Duration'] = days_val

In [16]:
df.update(df_hours)
df.update(df_minutes)
df.update(df_days)
df['Duration'] = df['Duration'].astype(float)

In [17]:
df

,Tour Name,Type,Rating,Number of reviews,Price,Duration
0,1 Day Private Mt Fuji Tour (Charter) - English...,Cable Car Tours,5.0,159,473.38,6.0
1,Mt Fuji and Hakone 1-Day Bus Tour return by Bu...,Audio Guides,5.0,1292,141.09,6.0
2,Tokyo Full-Day Private Tour with Government-Li...,Public Transportation Tours,5.0,571,152.04,6.0
3,"Mt Fuji, Hakone Lake Ashi Cruise Bullet Train ...",Bus Tours,4.0,652,133.94,6.0
4,Private Full Day Sightseeing Tour to Mount Fuj...,Bus Tours,5.0,286,647.98,6.0
...,...,...,...,...,...,...
979,"Private Lunch with Sayuki, the First Western G...",Transportation Services,NaN,NaN,NaN,2.0
980,Transfer in private vehicle from Tokyo City Ce...,Full-day Tours,NaN,NaN,NaN,1.5
981,Private Custom Day Tour in Tokyo City,Transportation Services,NaN,NaN,NaN,6.0
982,Tokyo Private Transfer from Tokyo central to N...,Food & Drink,NaN,NaN,NaN,1.0


In [18]:
df.rename(columns={'Duration': 'Duration (hrs)'}, inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tour Name          984 non-null    object 
 1   Type               983 non-null    object 
 2   Rating             485 non-null    float64
 3   Number of reviews  485 non-null    object 
 4   Price              948 non-null    float64
 5   Duration (hrs)     984 non-null    float64
dtypes: float64(3), object(3)
memory usage: 46.2+ KB


In [20]:
df.to_csv('tour_final.csv', index = False, header = True)